このファイル上で、訓練済みモデルを用いてMIDI生成を行い、モデルの性能を評価する

inputを何にするかは未定（txt, seed, midi...）

In [ ]:
#cuda　動作確認用
import torch


2.5.1+cu118
True


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu118
True


In [3]:
print(torch.cuda.device_count())

1


In [12]:
from music21 import converter

# MIDIファイルを読み込む
midi_file = converter.parse('./data/output/generated_output3.mid')

# 詳細な情報を出力
print(midi_file)

# 各音符の情報を表示
for element in midi_file.recurse().notesAndRests:
    print(element)

<music21.stream.Score 0x252ea803bf0>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note C#>
<music21.note.Note F#>
<music21.note.Note C#>
<music21.note.Note G>
<music21.note.Note G#>
<music21.note.Note A>
<music21.note.Note G#>
<music21.note.Note F#>
<music21.note.Note E>
<music21.note.Note G>
<music21.note.Note B>
<music21.note.Note B>
<music21.note.Note A>
<music21.note.Note E->
<music21.note.Note D>
<music21.note.Note E->
<music21.note.Note E>
<music21.note.Note B>
<music21.note.Note F#>
<music21.note.Note F#>
<music21.note.Note D>
<music21.note.Note B>
<music21.note.Note A>
<music21.note.Note D>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note A>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note C#>
<music21.note.Note E>
<music21.note.Note D>
<music21.note.Note E>
<music21.note.Note F#>
<music21.note.Note G>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Note C#>
<music21.note.Note D>
<mus

In [13]:
import mido

# MIDIファイルを開く
mid = mido.MidiFile('./data/midi_data/mz_311_1.mid')

# MIDIトラックの情報を表示
for i, track in enumerate(mid.tracks):
    print(f'Track {i}: ')
    for msg in track:
        print(msg.bytes())

Track 0: 
[255, 3, 34, 75, 108, 97, 118, 105, 101, 114, 115, 111, 110, 97, 116, 101, 32, 78, 114, 46, 32, 56, 32, 75, 86, 32, 51, 49, 49, 32, 49, 46, 32, 83, 97, 116, 122]
[255, 2, 35, 67, 111, 112, 121, 114, 105, 103, 104, 116, 32, 169, 32, 50, 48, 48, 54, 32, 118, 111, 110, 32, 66, 101, 114, 110, 100, 32, 75, 114, 252, 103, 101, 114, 46, 32]
[255, 1, 23, 87, 111, 108, 102, 103, 97, 110, 103, 32, 65, 109, 97, 100, 101, 117, 115, 32, 77, 111, 122, 97, 114, 116]
[255, 1, 27, 72, 101, 114, 97, 117, 115, 103, 101, 103, 101, 98, 101, 110, 32, 97, 109, 32, 32, 49, 46, 55, 46, 50, 48, 48, 54, 10]
[255, 1, 19, 85, 112, 100, 97, 116, 101, 32, 97, 109, 32, 53, 46, 54, 46, 50, 48, 49, 51, 10]
[255, 1, 21, 68, 97, 117, 101, 114, 58, 32, 32, 52, 58, 49, 57, 32, 77, 105, 110, 117, 116, 101, 110, 10]
[255, 88, 4, 4, 2, 24, 8]
[255, 89, 2, 2, 0]
[255, 81, 3, 6, 116, 130]
[255, 6, 6, 84, 101, 105, 108, 32, 49]
[255, 81, 3, 6, 49, 112]
[255, 81, 3, 6, 189, 129]
[255, 81, 3, 6, 38, 201]
[255, 81, 3, 6, 